## A esta función dale el path adecuado y te va a devolver listas

In [24]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path_fig = '/home/nico/Documentos/LatLon/Linieros/P2/Figuras/'

In [2]:
data = {#revisado
    'Nombre': ['Iquique Los Cóndores', 'Iquique Diego Aracena', 'Calama el Loa','Antofagasta Cerro Moreno','UNIV CATÓLICA DEL NORTE ANTOFAGASTA ','CHAÑARAL','Copiapó Chamonate', 'DESIERTO DE ATACAMA CALDERA','UNIV DE ATACAMA COPIAPÓ','FREIRINA NICOLASA','LA SERENA LA FLORIDA','OVALLE','CERRO TOLOLO','VIÑA DEL MAR TORQUEMADA','CRISTO REDENTOR','RODELILLO','TOBALABA','SANTIAGO QUINTA NORMAL','SANTIAGO PUDAHUEL','SANTO DOMINGO'],
    'Código': [          200005,               200006,               220002,      230001                  ,230002,                                 260002,     270002   ,270008,                                 270009           ,280010                      ,290004       ,300024   ,300034       ,320041                  ,320051           ,330007     ,330019    ,330020                  ,330021             ,330030]
}
aws = pd.DataFrame(data)

def Goodwin_model(PP,V,Vd):
    # Goodwin et al. model 1983, assumes that all the drops collected freeze on the cable. In other words, the growth mode is dry. 
    # Goodwin et al. model is conceptually correct, good predictions at moderate conditions and fair predictions in extreme conditions. More in Makkonen, Lasse (1998) Modeling power line icing in freezing precipitation
    rho_w =  997 #kg/m3
    rho_i =  917 #kg/m3
    DR = ((rho_w*PP)/(np.pi*rho_i))*np.sqrt(1+(V/Vd)) # Unidades de mm
    return DR

def Calc_Vd(PP_intensidad):
    # Calculo radio promedio del tamaño de gota r_0 detalles en Sheng 2023, "Estimating and Mapping Extreme Ice Accretion Hazard and Load Due to Freezing Rain at Canadian Sites" 
    r_0 = 1.835/(4.1*PP_intensidad**(-0.21))# -> Marshall-Palmer drop size distribution (Atlas et al. 1973)
    print(f"El radio promedio de la distribución es de: {r_0} [mm]")
    if r_0 < 0.6:
        print("r_0 < 0.6")
        Vd = 8*r_0
    elif r_0 > 0.6 and r_0 < 2.0:
        print("r_0 > 0.6 and r_0 < 2.0")
        Vd = 201*(r_0/1000)**0.5
    print(f"El valor de la velocidad de caida de gota es Vd = {Vd} [m/s]") 
    return Vd   

# Modelo de crecimiento de hielo en conductores 

In [29]:
# Modelo de acreción de hielo en conductores
# La función Goodwin_model(PP, V, Vd) calcula los milimetros de hielo en un conductor, tiene como entradas:
# PP precipitación en mm
# V velocidad del viento en m/s
# Vd velocidad de caída de gota

# La función Calc_Vd calcula la velocidad de caída de una gota en función de la intensidad de la PP utilizando la distribución de Marshall-Palmer

def Goodwin_model(PP, V, Vd):
    # Goodwin et al. model 1983, assumes that all the drops collected freeze on the cable. In other words, the growth mode is dry. 
    # Goodwin et al. model is conceptually correct, good predictions at moderate conditions and fair predictions in extreme conditions. More in Makkonen, Lasse (1998) Modeling power line icing in freezing precipitation
    rho_w =  997 #kg/m3
    rho_i =  917 #kg/m3
    DR = ((rho_w * PP) / (np.pi * rho_i)) * np.sqrt(1 + (V / Vd)) # Unidades de mm
    print(f"El espezor calculado es de {round(DR.max(),1)}")
    return DR

def Calc_Vd(PP_intensidad):
    # Calculo radio promedio del tamaño de gota r_0 detalles en Sheng 2023, "Estimating and Mapping Extreme Ice Accretion Hazard and Load Due to Freezing Rain at Canadian Sites" 
    r_0 = 1.835 / (4.1 * PP_intensidad ** (-0.21))  # -> Marshall-Palmer drop size distribution (Atlas et al. 1973)
    #print(f"El radio promedio de la distribución es de: {r_0} [mm]")
    Vd = np.where(r_0 < 0.6, 8 * r_0, np.where((r_0 > 0.6) & (r_0 < 2.0), 201 * (r_0 / 1000) ** 0.5, np.nan))
    #print(f"El valor de la velocidad de caída de gota es Vd = {Vd} [m/s]") 
    return Vd

In [3]:
def read_csv_files(folder_path):
    dataframes = []
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".csv"):
            file_path = os.path.join(folder_path, file)
            df = pd.read_csv(file_path)
            dataframes.append(df)
    return dataframes

In [4]:
def limpieza(df,columna):
    df[columna] = df[columna][df[columna] != '.']
    df = df.dropna()
    df = df.reset_index(drop=True)
    df[columna]=df[columna].astype(float)
    return df
def localiza_columna(df, columna, valor):
    return df.loc[df[columna] == valor]


In [5]:
#Leo los datos de PP, VV y TT
folder_path = "/home/nico/Documentos/LatLon/Linieros/P2/mediciones/Precipitacion acumulada a 6 horas"
dfs_PP = read_csv_files(folder_path)
folder_path = "/home/nico/Documentos/LatLon/Linieros/P2/mediciones/Viento a 10 metros de altura"
dfs_VV = read_csv_files(folder_path)
folder_path = "/home/nico/Documentos/LatLon/Linieros/P2/mediciones/Temperatura"
dfs_TT = read_csv_files(folder_path)

/tmp/ipykernel_287223/175095243.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [7]:
dfs_PP[0].head()

,Codigo_estacion,Fecha_medicion,Precipitacion,Valor_Traza
0,200005,01-01-1952 06:00,.,.
1,200005,01-01-1952 12:00,0.0,Falso
2,200005,01-01-1952 18:00,.,.
3,200005,02-01-1952 00:00,0.0,Falso
4,200005,02-01-1952 06:00,.,.


In [8]:
dfs_TT[0].head()

,Codigo_estacion,Fecha_medicion,Temperatura_aire
0,200005,01-01-1952 06:00,14.4
1,200005,01-01-1952 07:00,13.8
2,200005,01-01-1952 08:00,14.8
3,200005,01-01-1952 09:00,16.2
4,200005,01-01-1952 10:00,18.0


In [9]:
dfs_VV[0].head()

,Codigo_estacion,Fecha_medicion,Orientacion_del_viento,Velocidad_viento_en_nudos,VRB,Velocidad_viento_en_metros
0,200005,01-01-1952 06:00,90,5.0,Falso,2.572281
1,200005,01-01-1952 07:00,90,2.0,Falso,1.028912
2,200005,01-01-1952 08:00,90,5.0,Falso,2.572281
3,200005,01-01-1952 09:00,0,0.0,Falso,0.000000
4,200005,01-01-1952 10:00,270,5.0,Falso,2.572281


## Unimos los df

In [10]:
dfs_PP[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16085 entries, 0 to 16084
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Codigo_estacion  16085 non-null  int64 
 1   Fecha_medicion   16085 non-null  object
 2   Precipitacion    16085 non-null  object
 3   Valor_Traza      16085 non-null  object
dtypes: int64(1), object(3)
memory usage: 502.8+ KB


In [34]:
path_fig = '/home/nico/Documentos/LatLon/Linieros/P2/Figuras/PP_congelante/'
for i, df in enumerate(dfs_PP):
    nom = aws['Nombre'][aws['Código'] == df['Codigo_estacion'][0]].values[0] #Extraigo el nombre de la estación
    print(f"Codigo de la estación {df['Codigo_estacion'][0]}")
    #print(f"Indice {i}")
    print(f"Nombre de la estación {nom}")
    df = limpieza(df,'Precipitacion')
    dfs_TT[i] = limpieza( dfs_TT[i],'Temperatura_aire')
    #df['Fecha_medicion'] = pd.to_datetime(df['Fecha_medicion'])
    
    #Merge de 3 df en dos pasos
    merged_df = pd.merge(df, dfs_VV[i] , on='Fecha_medicion', how='inner')
    merged_df = pd.merge(merged_df, dfs_TT[i] , on='Fecha_medicion', how='inner')
    
    #1- Elimino filas que no tengan mediciones de PP
    #2- Elimino filas con mediciones de temperatura positivos
    merged_df = merged_df[merged_df['Precipitacion'] != 0.0]# Elimino las filas sin PP
    merged_df = merged_df[merged_df['Temperatura_aire'] < 0]# Elimino las filas con temp mayores que 0ºC
    
    #Corremos el Modelo de Goodwin, primero calculo la velocidad de la gota en función de la PP
    merged_df['Vd_drop_vel'] = Calc_Vd(merged_df['Precipitacion'])
    merged_df['Ice_mm'] = Goodwin_model(merged_df['Precipitacion'], merged_df['Velocidad_viento_en_nudos'], merged_df['Vd_drop_vel'])
    
    codigo = df['Codigo_estacion'][0]
    
    ## Figura
    #plt.figure(figsize=(8, 6))
    #scatter = plt.scatter(merged_df['Temperatura_aire'],merged_df['Precipitacion'],25,merged_df['Velocidad_viento_en_nudos']/1.9438, cmap='viridis')
    #colorbar = plt.colorbar(scatter)  # Agrega el colorbar
    #colorbar.set_label('Viento [m/s]')
    ## Etiquetas y título
    #plt.title(f"{nom}, código = {df['Codigo_estacion'][0]}", fontsize=12)
    #plt.xlabel('Temperatura [ºC]', fontsize=12)
    #plt.ylabel('Precipitación [mm]', fontsize=12)
    #plt.savefig(path_fig+str(codigo)+'_'+nom+'_scatter_Temp_PP_VV_negativosPP', dpi=300)
    #plt.show()
    #plt.close()
    print('########################')
    
    

Codigo de la estación 200005
Nombre de la estación Iquique Los Cóndores
El espezor calculado es de nan
########################
Codigo de la estación 200006
Nombre de la estación Iquique Diego Aracena
El espezor calculado es de nan
########################
Codigo de la estación 220002
Nombre de la estación Calama el Loa
El espezor calculado es de 2.8
########################
Codigo de la estación 230001
Nombre de la estación Antofagasta Cerro Moreno
El espezor calculado es de nan
########################
Codigo de la estación 230002
Nombre de la estación UNIV CATÓLICA DEL NORTE ANTOFAGASTA 
El espezor calculado es de nan
########################
Codigo de la estación 260002
Nombre de la estación CHAÑARAL
El espezor calculado es de nan
########################
Codigo de la estación 270002
Nombre de la estación Copiapó Chamonate
El espezor calculado es de nan
########################
Codigo de la estación 270008
Nombre de la estación DESIERTO DE ATACAMA CALDERA
El espezor calculado es de

In [ ]:
path_fig = '/home/nico/Documentos/LatLon/Linieros/P2/Figuras/'
for i, df in enumerate(dfs_PP):
    print(f"Codigo de la estación {df['Codigo_estacion'][0]}")
    print(f"Indice {i}")
    df = limpieza(df,'Precipitacion')
    #df['Fecha_medicion'] = pd.to_datetime(df['Fecha_medicion'])
    merged_df = pd.merge(df, dfs_VV[i] , on='Fecha_medicion', how='inner')
    merged_df = pd.merge(merged_df, dfs_TT[i] , on='Fecha_medicion', how='inner')
    
    merged_df = merged_df[merged_df['Precipitacion'] != 0.0]# Elimino las filas sin PP
    merged_df = merged_df[merged_df['Temperatura_aire'] < 1]# Elimino las filas con temp menores que 1ºC
    
    
    ## ID y nmombre de la estación
    nom = aws['Nombre'][aws['Código'] == df['Codigo_estacion'][0]].values[0] #Extraigo el nombre de la estación
    codigo = df['Codigo_estacion'][0]
    ## Figura
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(merged_df['Temperatura_aire'],merged_df['Precipitacion'],25,merged_df['Velocidad_viento_en_nudos']/1.9438, cmap='viridis')
    colorbar = plt.colorbar(scatter)  # Agrega el colorbar
    colorbar.set_label('Viento [m/s]')
    # Etiquetas y título
    plt.title(f"{nom}, código = {df['Codigo_estacion'][0]}", fontsize=12)
    plt.xlabel('Temperatura [ºC]', fontsize=12)
    plt.ylabel('Precipitación [mm]', fontsize=12)
    plt.savefig(path_fig+str(codigo)+'_'+nom+'_scatter_Temp_PP_VV', dpi=300)
    plt.show()
    plt.close()
    

In [19]:
merged_df

,Codigo_estacion_x,Fecha_medicion,Precipitacion,Valor_Traza,Codigo_estacion_y,Orientacion_del_viento,Velocidad_viento_en_nudos,VRB,Velocidad_viento_en_metros,Codigo_estacion,Temperatura_aire


In [14]:
'''
plt.figure(figsize=(8, 6))
scatter = plt.scatter(merged_df['Temperatura_aire'],merged_df['Precipitacion'],25,merged_df['Velocidad_viento_en_nudos']/1.9438, cmap='viridis')
colorbar = plt.colorbar(scatter)  # Agrega el colorbar
colorbar.set_label('Viento [m/s]')

# Etiquetas y título
plt.title(f"{nom}, código = {df['Codigo_estacion'][0]}", fontsize=12)
plt.xlabel('Temperatura [ºC]', fontsize=12)
plt.ylabel('Precipitación [mm]', fontsize=12)
plt.savefig(path_fig+str(codigo)+'_scatter_Temp_PP_VV', dpi=300)

'''


'\nplt.figure(figsize=(8, 6))\nscatter = plt.scatter(merged_df[\'Temperatura_aire\'],merged_df[\'Precipitacion\'],25,merged_df[\'Velocidad_viento_en_nudos\']/1.9438, cmap=\'viridis\')\ncolorbar = plt.colorbar(scatter)  # Agrega el colorbar\ncolorbar.set_label(\'Viento [m/s]\')\n\n# Etiquetas y título\nplt.title(f"{nom}, código = {df[\'Codigo_estacion\'][0]}", fontsize=12)\nplt.xlabel(\'Temperatura [ºC]\', fontsize=12)\nplt.ylabel(\'Precipitación [mm]\', fontsize=12)\nplt.savefig(path_fig+str(codigo)+\'_scatter_Temp_PP_VV\', dpi=300)\n\n'

In [35]:
aws


,Nombre,Código
0,Iquique Los Cóndores,200005
1,Iquique Diego Aracena,200006
2,Calama el Loa,220002
3,Antofagasta Cerro Moreno,230001
4,UNIV CATÓLICA DEL NORTE ANTOFAGASTA,230002
5,CHAÑARAL,260002
6,Copiapó Chamonate,270002
7,DESIERTO DE ATACAMA CALDERA,270008
8,UNIV DE ATACAMA COPIAPÓ,270009
9,FREIRINA NICOLASA,280010
